In [67]:
import torch
import torch.nn as nn
from torchvision import transforms
from torchvision import models
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from datasets import Dataset, load_from_disk

import numpy as np
import pandas as pd

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


In [3]:
path_final_test = './data/final_test/'

In [7]:
BATCH_SIZE = 128

In [4]:
preprocess = transforms.Compose([transforms.Resize(256), transforms.CenterCrop(224), transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406],  std=[0.229, 0.224, 0.225])])

In [5]:
test_dataset = ImageFolder(path_final_test, preprocess)

In [6]:
test_dataset

Dataset ImageFolder
    Number of datapoints: 1751
    Root location: ./data/final_test/
    StandardTransform
Transform: Compose(
               Resize(size=256, interpolation=bilinear, max_size=None, antialias=True)
               CenterCrop(size=(224, 224))
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [8]:
testDataLoader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [12]:
next(iter(testDataLoader))[1]

tensor([0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1,
        0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
        0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1,
        1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
        0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1,
        1, 0, 1, 0, 1, 1, 1, 1])

In [13]:
resnet34 = models.resnet34(weights=None)
class CustomFC(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.fc1 = nn.Linear(in_features=in_features, out_features=out_features)
        
    def forward(self, x):
        return self.fc1(x), x
    
resnet34.fc = CustomFC(512,8)
resnet34.to(device)

resnet34.load_state_dict(torch.load('model_best_resnet34.pt', map_location=device, weights_only=False))

<All keys matched successfully>

In [68]:
def naive_mod_acc(model, dataloader_test):
    model.eval()
    trg = []
    pred = []
    acc_test = 0.0
    tot_test = 0.0
    big_ones = 0.0
    tot_positive = 0.0
    with torch.no_grad():
        for itms in dataloader_test:
            x=itms[0]
            target=itms[1]
            x = x.to(device)
            target = target.to(device)
            outputs, _ = model(x)
            outputs = outputs.sigmoid().max(dim=1).values
            # print(outputs.shape)
            acc_test += torch.sum(outputs.round() == target).item()
            big_ones += torch.sum(outputs.round()).item()
            pred.extend(outputs.round().cpu().tolist())
            trg.extend(target.cpu().tolist())
            tot_test += target.size(0)
            tot_positive += torch.sum(target).item()
        print(f'Test accuracy: {round(acc_test/tot_test*100,2)}%')
        print(f'Higher than 50%: {round(big_ones/tot_test*100,2)}%')
        print(f'Non-anomalies to total: {round(tot_positive/tot_test*100,2)}%')
        
    return round(acc_test/tot_test*100,2), round(big_ones/tot_test*100,2), pd.DataFrame({'true':trg, 'pred':pred})

In [70]:
_,_, res = naive_mod_acc(resnet34, testDataLoader)
res

Test accuracy: 53.85%
Higher than 50%: 92.69%
Non-anomalies to total: 48.03%


,true,pred
0,0,1.0
1,1,1.0
2,0,1.0
3,1,1.0
4,0,1.0
...,...,...
1746,1,1.0
1747,1,1.0
1748,0,1.0
1749,0,1.0
